In [1]:
module TSP2dim

using LinearAlgebra
using Combinatorics
using Plots
gr()

export TSG, DEPOT, visualize

const DEPOT = [0.0, 0.0]

struct TSG
    points::Array{Float64, 2}
end

function visualize(game)
    f = plot(size=(500, 500), aspect=:equal)
    scatter!(f, [DEPOT[1]], [DEPOT[2]], color=:red, label=nothing)
    scatter!(f, game.points[:, 1], game.points[:, 2], color=:blue, marker=:rect, label=nothing)
    f
end


"""
    naive computation of the Shapley values of the given TSG
"""
function naive(game)
    n = size(game.points, 1)
    values = zeros(n)
    
    # distance matrix
    Ddepot = zeros(n)
    D = zeros(n, n)
    for i in 1:n
        Ddepot[i] = norm(DEPOT .- game.points[i, :])
    end
    for i in 1:n, j in i+1:n
        D[i, j] = D[j, i] = norm(game.points[i, :] .- game.points[j, :])
    end
    
    # cost pre-processing for all subsets on {1,...,n}
    dictcost = Dict()
    dictcost[[]] = 0.0
    for N in powerset(collect(1:n), 1)
        if length(N) == 1
            dictcost[N] = 2 * Ddepot[N[1]]
        else
            mincost = Inf
            for perm in permutations(N)
                cost = 0.0
                for i in 1:length(perm) - 1
                    cost += D[perm[i], perm[i + 1]]
                end
                cost += Ddepot[perm[1]]
                cost += Ddepot[perm[end]]
                if mincost > cost
                    mincost = cost
                end
            end
            dictcost[N] = mincost
        end
    end
    
    # computation
    for i in 1:n
        Ns = [j for j in 1:n if j != i]
        valuei = 0.0
        for S in powerset(Ns)
            Si = sort([S; i])
            c1 = dictcost[Si]
            c2 = dictcost[S]
            valuei += factorial(length(S)) * factorial(n - length(S) - 1) / factorial(n) * (c1 - c2)
        end
        values[i] = valuei
    end
    values
end


"""
    naive computation of the Shapley values of the given TSG
"""
function fast_with_sc(game)
    n = size(game.points, 1)
    values = zeros(n)
    
    # distance matrix
    Ddepot = zeros(n)
    D = zeros(n, n)
    for i in 1:n
        Ddepot[i] = norm(DEPOT .- game.points[i, :])
    end
    for i in 1:n, j in i+1:n
        D[i, j] = D[j, i] = norm(game.points[i, :] .- game.points[j, :])
    end
    
    S = zeros(n, n)
    for i in 1:n, j in i:n
        S[i, j] = S[j, i] = Ddepot[i] + Ddepot[j] - D[i, j]
    end
    # println(S)
    

    # trial Φ1
    # v1 = S[1, 1] - S[1, 2] / 2
    # v2 = S[2, 2] - S[1, 2] / 2
    # println(v1)
    # println(v2)
    
    for k in 1:n
        values[k] = S[k, k]
        vv = [S[j, k] for j in 1:n if j != k]
        sort!(vv, rev=true)
        sc = 0.0
        for l in 1:length(vv)
            sc += vv[l] / (l * (l + 1))
        end
        values[k] -= sc
    end
    values
end


end

using Main.TSP2dim

In [2]:
tsg = TSG([4.0 1.0; 1.0 1.0])
TSP2dim.fast_with_sc(tsg)

2-element Vector{Float64}:
 6.977551657239943
 1.5597675307508125

In [3]:
tsg = TSG([4.0 1.0; 1.0 1.0])
TSP2dim.naive(tsg)

2-element Vector{Float64}:
 6.977551657239943
 1.5597675307508123

In [4]:
tsg = TSG([4.0 1.0; 1.0 1.0; 0.5 2.0])
TSP2dim.naive(tsg)

3-element Vector{Float64}:
 6.491160705824171
 1.1668121330121404
 2.4578484809858727

In [5]:
TSP2dim.fast_with_sc(tsg)

3-element Vector{Float64}:
 6.551022973010413
 1.1668121330121408
 2.457848480985873